In [1]:
import numpy as np

# Known parameters
RA = 0.5         # Inner radius (m)
RB = 1.0         # Outer radius (m)
lambda_ = 25.0   # Thermal conductivity (W/m·K)
h_air = 100.0    # Heat transfer coefficient (W/m^2·K)
T_air = 298.15   # Ambient temperature (K)
T0 = 373.15      # Temperature at r = RA (K)
N = 5            # Number of nodes
L = 1.0          # Axial length (m) for the cylinder

# Radial grid (node-centered): i=0 at r=RA, i=N-1 at r=RB
r = np.linspace(RA, RB, N)

# System matrix A and right-hand side b
A = np.zeros((N, N))
b = np.zeros(N)

# Helper function for the radial conduction coefficient 
def radial_conduct_coeff(r_i, r_ip1, k=lambda_, length=L):
    return 2.0 * np.pi * k * length / np.log(r_ip1 / r_i)

# 1) Dirichlet boundary at i=0: T(0) = T0
#    We force T[0] = T0 by setting A[0,0] = 1, b[0] = T0
A[0, 0] = 1.0
b[0] = T0

# 2) Convection boundary at i=N-1: 
#    conduction from node (N-2) -> (N-1) 
 
k_edge = radial_conduct_coeff(r[N-2], r[N-1])
A[N-1, N-1] = +k_edge + h_air * 2.0 * np.pi * r[N-1] * L
A[N-1, N-2] = -k_edge
b[N-1] = h_air * 2.0 * np.pi * r[N-1] * L * T_air

# 3) Interior nodes (i=1,...,N-2): conduction balance
 
for i in range(1, N-1):
    k_left = radial_conduct_coeff(r[i-1], r[i])
    k_right = radial_conduct_coeff(r[i], r[i+1])
    A[i, i-1] = -k_left
    A[i, i] = k_left + k_right
    A[i, i+1] = -k_right

# Solve the linear system A*T = b
T_sol = np.linalg.solve(A, b)

# Print final temperatures
for i in range(N):
    print('Node', i, 'at r=', r[i], 'm: T=', 
		f'{T_sol[i]:.2f}', 'K')


Node 0 at r= 0.5 m: T= 373.15 K
Node 1 at r= 0.625 m: T= 355.41 K
Node 2 at r= 0.75 m: T= 340.91 K
Node 3 at r= 0.875 m: T= 328.65 K
Node 4 at r= 1.0 m: T= 318.03 K
